$$\newcommand{\nr}[1]{\|#1\|}
\newcommand{\RR}{\mathbb{R}}
\newcommand{\N}{\mathbb{N}}
$$
### MEU352 2023/2024 - Analyse numérique matricielle et optimisation

# TP1 - Résolution de systèmes linéaires triangulaires. Méthode de Gauss.



## Exercice 0. Manipulation de vecteurs et de matrices.

On aura besoin des modules de python ``numpy`` et ``matplotlib.pyplot``. On peut les charger en exécutant les commandes

``import numpy as np``

``import matplotlib.pyplot as plt``

(on désignera alors le module ``numpy`` par ``np`` et ``matplotlib.pyplot`` par ``plt``. 

**Q1.** Executez les commandes suivantes et affichez le résultat. Essayez de comprendre ce que vous avez obtenu.

``
u = np.array([1,2,3,4,5])
v = np.array([[1,2,3,4,5]])
su=u.shape
sv=v.shape
ut = np.transpose(u)
vt = np.transpose(v)
vt2 = np.array([[1],[2],[3],[4],[5]])
A = np.array([[1,2,0,0,0],[0,0,2,3,1],[0,0,0,2,2],[0,0,0,0,1],[1,1,1,0,0]])
B = np.array([[1,2,3,4,5],[2,3,4,5,6],[3,4,5,6,7],[4,5,6,7,8],[5,6,7,8,9]])
d=np.diag(A)
dd=np.array([np.diag(A)])
dt=np.transpose(d)
ddt=np.transpose(dd)
Ad=np.diag(np.diag(A))``

**Q2.** Même question pour les commandes suivantes.

``u*v,  u*vt, vt*u, u/v, u/vt, v/v, v/vt, np.vdot(u,v), np.vdot(u,vt)``

``A*B, np.dot(A,B)``

``np.dot(A,u), np.dot(A,v), np.dot(v,A), np.dot(A,vt), np.linalg.inv(A), np.dot(np.linalg(inv(A)),A))``

In [1]:
u = np.array([1,2,3,4,5])
v = np.array([[1,2,3,4,5]])
su=u.shape
sv=v.shape
ut = np.transpose(u)
vt = np.transpose(v)
vt2 = np.array([[1],[2],[3],[4],[5]])
A = np.array([[1,2,0,0,0],[0,2,0,0,0],[0,0,3,0,0],[0,0,0,4,0],[0,0,0,0,5]])
B = np.array([[1,2,3,4,5],[2,3,4,5,6],[3,4,5,6,7],[4,5,6,7,8],[5,6,7,8,9]])
d=np.diag(A)
dd=np.array([np.diag(A)])
dt=np.transpose(d)
ddt=np.transpose(dd)
Ad=np.diag(np.diag(A))

## Exercice 1.  Résolution d'un système linéaire triangulaire.

Soit $A\in\mathcal{M}_n(\RR)$ une matrice triangulaire inférieure inversible, de taille $n\in\N,$ et $b\in\RR^n$. Comme $A$ est triangulaire inférieure, on peut résoudre le système $Ax=b$ par une technique dite de *descente* : la solution $x=(x_1,\dots,x_n)$ est obtenue en calculant successivement ses composantes $x_i$ par les formules
$$
\begin{align}
x_1&=\frac{b_1}{A_{11}}\\
x_2&=\frac{b_2-A_{21}\,x_1}{A_{22}}\\
&\vdots\\
x_n&=\frac{b_n-(A_{n1}\,x_1+\cdots +A_{n\,n-1}\,x_{n-1})}{A_{nn}}\\
&\\
&\\
\bigg(\,\,x_i&=\frac{b_i-(A_{i1}\,x_1+\cdots +A_{i\,i-1}\,x_{i-1})}{A_{ii}}\,\, \bigg)
\end{align}
$$

**Q1.** Définir une fonction ``descente`` qui prend en argument une matrice $A$ triangulaire inférieure inversible et un vecteur $b$ et qui retourne la solution $x$ du système $Ax=b$. Tester votre fonction sur une matrice $A$ à coefficients aléatoires et un second membre $b$ tel que la solution $x$ de $Ax=b$ soit connue.

**Q2.** Écrire la solution $x$ du système $Ax=b$ lorsque $A$ est cette fois-ci triangulaire supérieure, en fonction des coefficients de $A$ et de $b$, en résolvant successivement les équations depuis la dernière jusqu'à la première (on dit qu'on résout le système $Ax=b$ par *remontée*).

**Q3.** Modifier votre fonction ``descente`` en une fonction que vous appelerez ``remonte_descente`` qui permet la résolution du système $Ax=b$ lorsque $A$ est triangulaire inférieure ou triangulaire supérieure. Votre fonction devra tester si la matrice $A$ est triangulaire supérieure ou inférieure.

*Commandes python : essayez les commandes ``np.tril(A), np.triu(A), np.tril(A,k), np.triu(A,k)``, avec $k=1$ ou $k=-1$, et ``np.random.rand(n,n)``, avec $n\in\N$. La somme $(A_{i1}\,x_1+\cdots +A_{i\,i-1}\,x_{i-1})$ peut être vue comme un produit scalaire entre deux vecteurs, utiliser ``np.vdot`` pour le produit scalaire*. 

In [229]:
def remonte_descente(A,b):
    x = 0 * b
    n = len(b)
    if np.allclose(A,np.tril(A)):
        for i in range(n):
            x[i] = (b[i]- np.dot(A[i,:i], x[:i]))/A[i,i]
    elif np.allclose(A,np.triu(A)):
        for i in range(n-1,-1,-1):
            x[i] = (b[i] - np.dot(A[i,i+1:],x[i+1:]))/A[i,i]
    else : 
        raise(ValueError("A n'est pas triangulaire"))
    return x

In [230]:
n = 5
A = np.random.rand(n,n)
A = np.tril(A) + np.eye(n)
xex = np.array([1]*n,dtype = float)
b = np.dot(A,xex)
x = remonte_descente(A,b)

print(np.dot(x - xex,x-xex))

1.6023737137301802e-31


## Exercice 2.  La méthode de Gauss.


On suppose que $A$ est une matrice carrée inversible et qu'il est
possible d'appliquer la méthode de Gauss à la matrice $A$ et donc la transformer en une matrice triangulaire supérieure $U$
à coefficients diagonaux non nuls simplement en effectuant
successivement des opérations élémentaires sur les lignes du type
$L_i$ devient $L_i + \beta L_j$. On suppose donc que les pivots de
la méthode de Gauss sont tous non nuls. 

**Q1.** Vérifier que l'algorithme suivant permet de transformer une matrice donnée $A$ en une matrice triangulaire supérieure $U$ par la méthode de Gauss :


```
U = A   # on prend une copie qu'on écrasera
pour j = 0 à n-1
    pour i = j + 1 à n - 1
         beta = U(i,j)/U(j,j)                # U(j,j) est le pivot
         pour k = j  à  n -1
            U(i, k) = U(i,k) - beta * U(j,k) # ligne i devient ligne i - beta * ligne j
         fin k
    fin i
fin j
retourner U
```

**Q2.** Ecrire une fonction Python de la forme ```met_gauss(A)``` correspondant à cet algorithme.

*Remarque : vous pouvez écrire les commandes $U(i, k) = U(i,k) - \beta U(j,k)$, pour $k=j,\dots,n-1$, sans utiliser de boucle sur $k$, en écrivant le vecteur $(U(i,j),\dots,U(i,n-1))$ comme ``U[i,j:]``.*

**Q3.** Appliquer cette fonction aux matrices
$$
A=\left (
\begin{array}{ccc}
9 & 8 & 6 \\
7 & 6 & 12 \\
9 & 3 & 9
\end{array}
\right )
\qquad \mbox{ et } \qquad 
B=\left (
\begin{array}{cccc}
11 & 8 & 3 & 13 \\
 2 & 12 & 7 & 10 \\
 3 & 3 & 17 & 13 \\
 11 & 2 & 12 & 7
\end{array}
\right )
$$

Les réponses attendues sont respectivement
\begin{equation*}
A=\left (
\begin{array}{ccc}
9 & 8 & 6 \\
0 & -0.2222222 & 7.3333333 \\
0 & 0 & -162
\end{array}
\right )\qquad
 \mbox{ et }\qquad 
B=\left (
\begin{array}{cccc}
11 & 8 & 3 & 13 \\
 0 &   10.5454545 &  6.45454545 &  7.63636364\\
 0 & 0 & 15.6810345 &  8.86206897 \\
 0 & 0 & 0 &  -8.81693238
\end{array}
\right )
\end{equation*}

**Q4.** Adapter votre fonction ```met_gauss(A)``` en une fonction ```met_gauss_sys(A,b)``` de façon à que l'on puisse l'utiliser pour résoudre un système $Ax=b$, avec $b\in\RR^n$ donné. Pour cela, il faut le long de la méthode de Gauss faire les mêmes opérations sur la matrice $A$ et sur le second membre $b$. Cette fonction retournera la solution $x$ du système $Ax=b$ en écrivant le système triangulaire équivalent obtenu par la méthode de Gauss, et en résolvant ce système triangulaire avec la fonction ```remonte_descente```. La tester avec une matrice $A$ et un vecteur $b$ aléatoires par exemple.

In [227]:
def met_gauss(A):
    U = A   # on prend une copie qu'on écrasera
    n = len(A)
    for j in range(n):
        for i in range(j +1,n):
            beta = U[i,j]/U[j,j]                # U(j,j) est le pivot
            U[i,j:] = U[i,j:] - beta * U[j,j:] # ligne i devient ligne i - beta * ligne j
    return U

In [228]:
A = np.array([[9,8,6],[7,6,12],[9,3,9]],dtype = float)
B = np.array([[11,8,3,13],[2,12,7,10],[3,3,17,13],[11,2,12,7]],dtype = float)

print(met_gauss(A))
print(met_gauss(B))

[[   9.            8.            6.        ]
 [   0.           -0.22222222    7.33333333]
 [   0.            0.         -162.        ]]
[[11.          8.          3.         13.        ]
 [ 0.         10.54545455  6.45454545  7.63636364]
 [ 0.          0.         15.68103448  8.86206897]
 [ 0.          0.          0.         -8.81693238]]


In [240]:
def metgauss_sys(A,b):
    U = A   # on prend une copie qu'on écrasera
    V = b   # on prend une copie qu'on écrasera
    n,m = A.shape
    if n!= m:
        raise ValueError("Erreur de dimension : A n'est pas carré")
    elif n!=b.size:
        raise ValueError("Erreur de dimension : les lignes de A n'ont pas la même longueur que les colones de b")
    else:
        for j in range(n):
            for i in range(j +1,n):
                beta = U[i,j]/U[j,j]                # U(j,j) est le pivot
                U[i,j:] = U[i,j:] - beta * U[j,j:]  # ligne i de U devient ligne i - beta * ligne j
                V[i] = V[i] - beta * V[j]           # ligne i de V devient ligne i - beta * ligne j
    return remonte_descente(U,V)

In [242]:
n = 5
A = np.random.rand(n,n) + float(n) + np.eye(n)
b = np.random.rand(len(A))
x = metgauss_sys(A,b)

print(np.allclose(np.dot(A,x),b))

True


## La méthode de Gauss avec stratégie de pivot partiel.

En pratique, pour des questions de stabilité numérique, on a intérêt à choisir à l'étape $j$ un pivot $A_{k,j}$, avec $k\geq j$ tel que $|A_{k,j}|$ est maximal (car cela signifie diviser par la quantité la plus grande possible). À l'étape $j$ de la méthode de Gauss, on commence alors par choisir $p$ tel que $|A_{p,j}|=\max_{k\geq j}|A_{k,j}|$ et on échange les lignes $p$ et $j$ de $A$.

**Q5.** Créer une fonction ```met_gauss_pivot(A,b)``` qui permet la résolution du système $Ax=b$ en utilisant cette stratégie de choix de pivot. La tester avec le même exemple que dans la question précédente.

**Q6. (Comparaison des deux méthodes).** Pour $n=10,20,30,\dots,200$ :
* construire une matrice $A\in\mathcal{M}_n(\RR)$ aléatoire, un vecteur $x_{ex}\in\RR^n$ aléatoire et calculer $b=Ax_{ex}$ ;
* Résoudre le système $Ax=b$ (dont la solution est $x=x_{ex}$) par la méthode de Gauss avec et sans choix de pivot ;
* Calculer la norme $\|x-x_{ex}\|$ pour chacune des méthodes.

Comparer les résultats obtenus pour les deux méthodes. Vous pouvez représenter $\|x-x_{ex}\|$, ou, ce qu'est mieux, $\mathrm{log}(\|x-x_{ex}\|)$ en fonction de la taille $n$ de la matrice.


In [ ]:
def met_gauss_pivot(A,b):
    U = A   # on prend une copie qu'on écrasera
    V = b   # on prend une copie qu'on écrasera
    n,m = A.shape
    if n!= m:
        raise ValueError("Erreur de dimension : A n'est pas carré")
    elif n!=b.size:
        raise ValueError("Erreur de dimension : les lignes de A n'ont pas la même longueur que les colones de b")
    else:
        for j in range(n):
            for i in range(j +1,n):
                beta = U[i,j]/U[j,j]                # U(j,j) est le pivot
                U[i,j:] = U[i,j:] - beta * U[j,j:]  # ligne i de U devient ligne i - beta * ligne j
                V[i] = V[i] - beta * V[j]           # ligne i de V devient ligne i - beta * ligne j
    return remonte_descente(U,V)